## Project description

The project's goal is to prepare a report for a bank’s loan division. We need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. 

The bank already has some data on customers’ credit worthiness.

Our report will be considered when building the credit score of a potential customer. 
This credit score will be used to evaluate the ability of a potential borrower to repay their loan.


### Open the data file and have a look at the general information. 


In [1]:
# Loading all the libraries
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
from nltk.stem import SnowballStemmer
english_stemmer = SnowballStemmer('english')

# Load the data
logs = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [2]:
# Let's see how many rows and columns our dataset has
logs.shape



(21525, 12)

In [3]:
# let's print the first N rows
logs.head(20)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


### Description of the data

- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


There seem to be some issues with the data. 
1. First of all, there are missing values in the "days_employed" and "total_income" columns. 

2. Secondly, the "days_employed" column, which represents work experience in days, appears to have some problems.

* The values in this column are written as floats with six decimal places; 
* and some of them are negative, which does not make sense since days cannot be negative. 
* Additionally, there are some extremely large values in this column, which seems unrealistic for a person's work experience. 

3. In the "education" column there is inconsistency in the way the values are written, with some being in lowercase and some in uppercase. There is also an identified value of "some college," which does not provide a clear indication of the customer's educational level. 

4. The "purpose" column also has some issues, with some values being written in different ways, such as "to have a wedding" and "having a wedding," or "buy real estate" and "purchase of the house." 


In [4]:
# Get info on data
logs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Most of the columns have no missig values except for "days_employed" and "total_income" and its non-null values are exactly the same - 19351. As we saw in one row of the printed sample NaN in "days_employed" corresponds with NaN in "total_income". I'll explore if that is the case for all database.

In [5]:
# Let's look in the filtered table at the the first column with missing data
logs[logs['days_employed'].isna()].head(20)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


As was assumed earlier there is a connection between NaN in columns "days_employed" and "total_income". In the printed sample every row that has the "NaN" value in "days_employed" has also NaN value in "total_income".

In [6]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
logs[(logs['days_employed'].isna()) & (logs['total_income'].isna())].shape


(2174, 12)

Number of rows of the filtered dataframe exactly matches the number of missing values! Now I can say for sure that those missing values are the same.


In [7]:
missing_values_percentage = logs.days_employed.isna().sum()/len(logs)*100
print(round(missing_values_percentage, 2))

10.1


The missing data percentage is 10%. It is not that large amount of data but still notable and I wouldn't delete it. Sinse there is a possible connection between columns "days_employed" and "total_income" there could also be a reason for that connection. And understanding this reason could help me fill the missing values and therefore improve quality of the analysis. I think I need to explore this possibility. 

I will need to examine the columns that may be logically connected to the "days_employed" or "total_income" columns in order to identify any potential connections to the missing values. These columns include "dob_years," which describes the age of the individual, and "income_type," which indicates the type of employment. It is possible that there may be a connection between these columns and the missing values in "days_employed" and "total_income."

In [8]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values

logs_nan = logs[logs.days_employed.isna()]

In [9]:
# Checking distribution
logs_nan.income_type.value_counts()
round(logs_nan.income_type.value_counts(normalize=True), 2)

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

employee         0.51
business         0.23
retiree          0.19
civil servant    0.07
entrepreneur     0.00
Name: income_type, dtype: float64

As I can see most of the missind values in the days_employed or total_income columns have people who is occupied as an employee, businessman or retiree. I need to check distribution it whole dataset to see if this is the general distribution or something unique.


For now it would be presumptuous to draw any conclusions, I need to explore the dataset futher for possible reasons and connection if there are any. 

In [10]:
# Checking the distribution in the whole dataset
logs.income_type.value_counts()
round(logs.income_type.value_counts(normalize=True), 2)


employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

employee                       0.52
business                       0.24
retiree                        0.18
civil servant                  0.07
unemployed                     0.00
entrepreneur                   0.00
paternity / maternity leave    0.00
student                        0.00
Name: income_type, dtype: float64

The distribution of income types in the entire dataset is similar to the one with missing values, so there is no discernible pattern to analyze further. It may be necessary to conduct additional exploration of the dataset to gain a clearer understanding of the data.


In [11]:
# Check for other reasons and patterns that could lead to missing values
logs_nan.dob_years.unique().size
round(logs_nan.dob_years.value_counts(normalize=True), 2)


56

34    0.03
40    0.03
31    0.03
42    0.03
35    0.03
36    0.03
47    0.03
41    0.03
30    0.03
28    0.03
57    0.03
58    0.03
54    0.03
38    0.02
56    0.02
37    0.02
52    0.02
39    0.02
33    0.02
50    0.02
51    0.02
45    0.02
49    0.02
29    0.02
43    0.02
46    0.02
55    0.02
48    0.02
53    0.02
44    0.02
60    0.02
61    0.02
62    0.02
64    0.02
32    0.02
27    0.02
23    0.02
26    0.02
59    0.02
63    0.01
25    0.01
24    0.01
66    0.01
65    0.01
21    0.01
22    0.01
67    0.01
0     0.00
68    0.00
69    0.00
20    0.00
71    0.00
70    0.00
72    0.00
19    0.00
73    0.00
Name: dob_years, dtype: float64

In [12]:
logs.dob_years.unique().size
round(logs.dob_years.value_counts(normalize=True), 2)


58

35    0.03
40    0.03
41    0.03
34    0.03
38    0.03
42    0.03
33    0.03
39    0.03
31    0.03
36    0.03
44    0.03
29    0.03
30    0.03
48    0.02
37    0.02
50    0.02
43    0.02
32    0.02
49    0.02
28    0.02
45    0.02
27    0.02
56    0.02
52    0.02
47    0.02
54    0.02
46    0.02
58    0.02
57    0.02
53    0.02
51    0.02
59    0.02
55    0.02
26    0.02
60    0.02
25    0.02
61    0.02
62    0.02
63    0.01
64    0.01
24    0.01
23    0.01
65    0.01
66    0.01
22    0.01
67    0.01
21    0.01
0     0.00
68    0.00
69    0.00
70    0.00
71    0.00
20    0.00
72    0.00
19    0.00
73    0.00
74    0.00
75    0.00
Name: dob_years, dtype: float64

Upon closer examination of the "dob_years" column for potential reasons for missing values (as I did not find any in the "income_type" column), I noticed that the distribution in the original dataset and the filtered one is also similar - all unique values are distributed evenly without any noticeable differences between the two datasets.

It appears that the missing values may be accidental, possibly due to errors in the data acquisition process.

In [13]:
# Checking for other patterns
for i in logs:
    print(logs[logs.days_employed.isna()][i].value_counts())
    print('------------------NEXT-COLUMN-----------------------')

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64
------------------NEXT-COLUMN-----------------------
Series([], Name: days_employed, dtype: int64)
------------------NEXT-COLUMN-----------------------
34    69
40    66
31    65
42    65
35    64
36    63
47    59
41    59
30    58
28    57
57    56
58    56
54    55
38    54
56    54
37    53
52    53
39    51
33    51
50    51
51    50
45    50
49    50
29    50
43    50
46    48
55    48
48    46
53    44
44    44
60    39
61    38
62    38
64    37
32    37
27    36
23    36
26    35
59    34
63    29
25    23
24    21
66    20
65    20
21    18
22    17
67    16
0     10
68     9
69     5
20     5
71     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64
------------------NEXT-COLUMN-----------------------
secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college            

**Conclusions**

It appears that the missing values may be accidental, possibly due to errors in the data acquisition process. However, since the amount of missing data is significant (10%), I will not delete it but will instead try to fill it in with appropriate values.

To accurately fill in the missing values, I need to first transform the data as it has some issues. Specifically, I need to check for duplicates and remove them, standardize the formatting of values in certain columns (e.g., by converting them to all lowercase), and categorize certain columns for more accurate calculations.


## Data transformation

Let's go through each column to see what issues we may have in them

I begin with fixing educational information.

In [14]:
# Let's see all values in education column to check if and what spellings will need to be fixed
logs.education.value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

In [15]:
# Fix the registers if required
logs.education = logs.education.str.lower()

In [16]:
# Checking all the values in the column to make sure we fixed them
logs.education.value_counts()


secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

Let's check the data the `children` column

In [17]:
# Let's see the distribution of values in the `children` column
logs.children.value_counts()
round(logs.children.value_counts(normalize=True), 5)

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

 0     0.65733
 1     0.22383
 2     0.09547
 3     0.01533
 20    0.00353
-1     0.00218
 4     0.00190
 5     0.00042
Name: children, dtype: float64

I see there are some abnormalities in the children column - there are families whith 20 children and families with -1. As these values are less than 0,5% it would be safe to just delete them.

In [18]:
# fix the data
logs = logs[(logs.children != -1) & (logs.children != 20)]

In [19]:
# Checking the `children` column again to make sure it's all fixed
logs.children.value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

I'll check the data in the `days_employed` column.

In [20]:
# Find problematic data in `days_employed`
logs.days_employed.value_counts() 

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-201.643573     1
-7120.517564    1
-2146.884040    1
-881.454684     1
-3382.113891    1
Name: days_employed, Length: 19240, dtype: int64

In [21]:
logs[logs.days_employed < 0].shape
round(len(logs[logs.days_employed < 0])/len(logs.days_employed)*100, 2)

(15809, 12)

73.87

Wow! Almost 74% of negative values! That's a lot - we cannot drop it due to it's amount, and cannot analyse it in this form. 
I assume the error was made at some point along the pipeline and I decide to convert all the negative value into positive one.

In [22]:
# Address the problematic values, if they exist
logs.days_employed = logs.days_employed.abs()

In [23]:
# Check the result - make sure it's fixed
logs[logs.days_employed < 0].shape


(0, 12)

In the beginning I noted another possible problem - some extremely large values in this column, which seems unrealistic for a person's work experience. Let's see how big this problem could be.


In [24]:
#how can a person work for such a long time?
unreliable_work_time = 365*80 
logs[logs.days_employed > unreliable_work_time].shape
round(len(logs[logs.days_employed > unreliable_work_time])/len(logs.days_employed)*100, 2)

(3431, 12)

16.03

There is too much unreliable data (16%) to simply drop it, but it is also not possible to address this issue. 
I would say that the whole column is corrupted for some reason. I think the process of data entering and aquisition should be reviewed. It would be hard to use this column for the analisys.

Let's look at the subset that shows us cases of unreliable work time.

In [25]:
logs[logs.days_employed > unreliable_work_time].sort_values(by='days_employed')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20444,0,328728.720605,72,secondary education,1,widow / widower,2,F,retiree,0,15443.094,purchase of the house for my family
9328,2,328734.923996,41,bachelor's degree,0,married,0,M,retiree,0,20319.600,transactions with my real estate
17782,0,328771.341387,56,secondary education,1,married,0,F,retiree,0,10983.688,transactions with commercial real estate
14783,0,328795.726728,62,bachelor's degree,0,married,0,F,retiree,0,12790.431,buying my own car
7229,1,328827.345667,32,secondary education,1,civil partnership,1,F,retiree,0,19546.075,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
7794,0,401663.850046,61,secondary education,1,civil partnership,1,F,retiree,0,7725.831,wedding ceremony
2156,0,401674.466633,60,secondary education,1,married,0,M,retiree,0,52063.316,cars
7664,1,401675.093434,61,secondary education,1,married,0,F,retiree,0,20194.323,housing transactions
10006,0,401715.811749,69,bachelor's degree,0,unmarried,4,F,retiree,0,9182.441,getting an education


In the printed sample I can see that 'income_type' of people who's worktime is enormiosly big is retiree. I need to check if it is so through all dataset.

In [26]:
logs[logs.days_employed > unreliable_work_time].income_type.value_counts()

retiree       3429
unemployed       2
Name: income_type, dtype: int64

Almost everyone in this subset are retirees. Let's see distribution of our main dataset 'income_type'.

In [27]:
logs.income_type.value_counts()

employee                       11050
business                        5054
retiree                         3839
civil servant                   1453
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

3429 out of 3839 retirees have unreliable work time. But there was also retirees with missing values in column 'days_employed'. Let's check their number.

In [28]:
logs_nan_sample = logs[logs.days_employed.isna()]
logs_nan_sample.income_type.value_counts()

employee         1101
business          504
retiree           410
civil servant     146
entrepreneur        1
Name: income_type, dtype: int64

And the rest of our retirees (410) have missing values in the column days_employed

In [29]:
3839-3429-410

0

It should be noted that all of the retirees in our data have missing or unreliable information in the 'days_employed' column. While I am unable to address this issue, it will need to be taken into consideration in future analysis.



Let's now look at the client's age and whether there are any issues there.

In [30]:
# Check the `dob_years` for suspicious values and count the percentage
logs.dob_years.value_counts().sort_index()

0     100
19     14
20     51
21    110
22    183
23    252
24    263
25    356
26    406
27    490
28    501
29    543
30    536
31    556
32    506
33    577
34    597
35    614
36    553
37    531
38    595
39    572
40    603
41    603
42    592
43    510
44    543
45    494
46    469
47    480
48    536
49    505
50    509
51    446
52    483
53    457
54    476
55    441
56    482
57    457
58    461
59    441
60    376
61    353
62    351
63    268
64    263
65    194
66    183
67    167
68     99
69     83
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [31]:
logs.dob_years.value_counts(normalize=True).sort_index().head()

0     0.004672
19    0.000654
20    0.002383
21    0.005140
22    0.008551
Name: dob_years, dtype: float64

In this column we see a hundred of people who's age is 0. May be the applicants didn't provide their age, may be age was lost in the process of entering the data. But sinse its amount is not big - just a half of the percrent - it is safe to just delete it.

In [32]:
# Address the issues in the `dob_years` column, if they exist
logs = logs[logs.dob_years != 0]

In [33]:
# Check the result - make sure it's fixed
logs[logs.dob_years == 0].dob_years.count()

0


Now let's check the family_status column. 

In [34]:
# Let's see the values for the column
logs.family_status.value_counts()


married              12254
civil partnership     4139
unmarried             2783
divorced              1179
widow / widower        947
Name: family_status, dtype: int64

Now let's check the `gender` column.

In [37]:
# Let's see the values in the column
logs.gender.value_counts()

F      14083
M       7218
XNA        1
Name: gender, dtype: int64

In [38]:
# Address the problematic values
logs = logs[logs.gender != 'XNA']

In [39]:
# Check the result
logs[logs.gender == 'XNA'].gender.count()


0


Now let's check the `income_type` column.

In [40]:
# Let's see the values in the column
logs.income_type.value_counts()

employee                       10996
business                        5033
retiree                         3819
civil servant                   1447
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [41]:
logs_pivot_age_income = logs.pivot_table(index='dob_years', columns='income_type', values='debt',aggfunc='count')

In [42]:
logs_pivot_age_income.sort_values(by='dob_years').head(10)

income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,retiree,student,unemployed
dob_years,,,,,,,,
19,7.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN
20,16.0,2.0,33.0,NaN,NaN,NaN,NaN,NaN
21,34.0,7.0,69.0,NaN,NaN,NaN,NaN,NaN
22,64.0,6.0,111.0,NaN,NaN,1.0,1.0,NaN
23,66.0,11.0,175.0,NaN,NaN,NaN,NaN,NaN
24,81.0,19.0,161.0,NaN,NaN,1.0,NaN,NaN
25,108.0,27.0,221.0,NaN,NaN,NaN,NaN,NaN
26,119.0,37.0,248.0,NaN,NaN,2.0,NaN,NaN
27,142.0,37.0,307.0,1.0,NaN,3.0,NaN,NaN


I can see there are some very young retirees from the age of 22 and up. It is possible that someone may have retired due to health issues so I won't do anithing about it.


Let's see if we have any duplicates in our data.

In [45]:
# Checking duplicates
logs.duplicated().sum()


71

In [46]:
# Address the duplicates, if they exist
logs = logs.drop_duplicates().reset_index(drop=True)

In [47]:
# Last check whether we have any duplicates
logs.duplicated().sum()

0

In [48]:
# Check the size of the dataset that you now have after your first manipulations with it
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  int64  
 1   days_employed     19149 non-null  float64
 2   dob_years         21230 non-null  int64  
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  int64  
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  int64  
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  int64  
 10  total_income      19149 non-null  float64
 11  purpose           21230 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB



The dataset has changed a bit - in the beginning there were 21525 rows, now it is 21230. 295 rows was deleted which is 1.4% of data. 

# Missing values


There is two columns which have corresponding ID column:
* "education" - "education_id",
* "family_status" - "family_status_id".

Let's make a dictionnary out of dataframe in order to approach column's values by IDs, not by a string type value.

In [49]:
# Find the dictionaries
dict_education = logs[['education', 'education_id']]
dict_education = dict_education.drop_duplicates().reset_index(drop=True)
dict_education

,education,education_id
0,bachelor's degree,0
1,secondary education,1
2,some college,2
3,primary education,3
4,graduate degree,4


In [50]:
dict_family_status = logs[['family_status', 'family_status_id']]
dict_family_status = dict_family_status.drop_duplicates().reset_index(drop=True)
dict_family_status

,family_status,family_status_id
0,married,0
1,civil partnership,1
2,widow / widower,2
3,divorced,3
4,unmarried,4


### Restoring missing values in `total_income`

Columns "days_employed" and "total_income" have 10% of missing values that we need to adress. Each column have to be adressed separetly - we want to fill the missing values carefully in order not to mess up the data.

Let's start with "total_income". 

Firstly, we need to categorize column "dob_years" by dividing its values into categories:
* age up to 25 years old;
* from 26 to 45;
* from 46 to 65
* from 66 and up.


In [51]:
# Let's write a function that calculates the age category
def age_group(age):
    if age <= 25:
        return '<25'
    if 26 <=age<=45:
        return '26-45'
    if 46 <= age<=65:
        return '46-65'
    else:
        return '>66'
    

In [52]:
# Test if the function works
age_group(19)
age_group(28)
age_group(46)
age_group(69)

'<25'

'26-45'

'46-65'

'>66'

In [53]:
# Creating new column based on function
logs['age_group'] = logs.dob_years.apply(age_group)


In [54]:
# Checking how values in the new column
logs.age_group.value_counts()

26-45    10900
46-65     8404
<25       1226
>66        700
Name: age_group, dtype: int64


Income is often influenced by education and work experience. Let's examine the relationship between these factors.


Let's create a table that only has data without missing values. This data will be used to restore the missing values.

In [55]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
logs_without_nan = logs[logs.days_employed.notna()]
logs_without_nan.head(15)
logs_without_nan.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,26-45
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,26-45
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-45
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-45
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,26-45
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,26-45
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,46-65
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,26-45
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,26-45


<class 'pandas.core.frame.DataFrame'>
Int64Index: 19149 entries, 0 to 21229
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19149 non-null  int64  
 1   days_employed     19149 non-null  float64
 2   dob_years         19149 non-null  int64  
 3   education         19149 non-null  object 
 4   education_id      19149 non-null  int64  
 5   family_status     19149 non-null  object 
 6   family_status_id  19149 non-null  int64  
 7   gender            19149 non-null  object 
 8   income_type       19149 non-null  object 
 9   debt              19149 non-null  int64  
 10  total_income      19149 non-null  float64
 11  purpose           19149 non-null  object 
 12  age_group         19149 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.0+ MB


In [56]:
# Look at the mean values for income based on your identified factors
logs_without_nan.pivot_table(index='income_type', columns='age_group', values='total_income', aggfunc='mean')

age_group,26-45,46-65,<25,>66
income_type,,,,
business,32930.411643,33051.369974,25752.351209,33470.062000
civil servant,27442.064958,27499.138709,24510.974242,30992.299000
employee,26128.693783,26015.185750,22265.980563,26185.024870
entrepreneur,79866.103000,NaN,NaN,NaN
paternity / maternity leave,8612.661000,NaN,NaN,NaN
retiree,24408.993847,22343.926661,14298.976000,19663.470405
student,NaN,NaN,15712.260000,NaN
unemployed,21014.360500,NaN,NaN,NaN


In [57]:
# Look at the median values for income based on your identified factors
logs_without_nan.pivot_table(index='income_type', columns='age_group', values='total_income', aggfunc='median')

age_group,26-45,46-65,<25,>66
income_type,,,,
business,28094.4310,28406.377,22814.5995,29314.4045
civil servant,24368.0150,23847.285,22758.5535,26089.6870
employee,23066.1730,22781.846,20634.6650,24643.1985
entrepreneur,79866.1030,NaN,NaN,NaN
paternity / maternity leave,8612.6610,NaN,NaN,NaN
retiree,20028.7250,19420.007,14298.9760,17074.5790
student,NaN,NaN,15712.2600,NaN
unemployed,21014.3605,NaN,NaN,NaN



I have created a tables that display the mean and median income for various age groups and income types. I would like to add education and gender as variables in the table to see if these factors also have an impact on income. To increase the accuracy of the analysis, I plan to use median values in the table so the outcome wouldn't be affected by outliers.

In [58]:
logs_without_nan.pivot_table(index='income_type', columns='education', values='total_income', aggfunc='median')

education,bachelor's degree,graduate degree,primary education,secondary education,some college
income_type,,,,,
business,32285.664,NaN,21887.825,25441.230,28778.744
civil servant,27564.459,17822.757,23734.287,21864.475,25694.775
employee,26587.423,31771.321,20159.186,21841.813,24209.430
entrepreneur,79866.103,NaN,NaN,NaN,NaN
paternity / maternity leave,NaN,NaN,NaN,8612.661,NaN
retiree,23030.247,28334.215,16415.785,18372.071,19221.903
student,15712.260,NaN,NaN,NaN,NaN
unemployed,32435.602,NaN,NaN,9593.119,NaN


In [59]:
logs_without_nan.pivot_table(index=['age_group', 'gender', 'income_type'], 
                             columns='education', 
                             values='total_income', 
                             aggfunc='median')

education                                     bachelor's degree  \
age_group gender income_type                                      
26-45     F      business                            30659.2985   
                 civil servant                       23774.2605   
                 employee                            24516.1440   
                 entrepreneur                        79866.1030   
                 paternity / maternity leave                NaN   
                 retiree                             32256.0055   
                 unemployed                          32435.6020   
          M      business                            37985.0310   
                 civil servant                       35708.1140   
                 employee                            31500.2210   
                 retiree                             24160.4520   
                 unemployed                                 NaN   
46-65     F      business                            31831.1920   
                 civil servant                       28015.9025   
                 employee                            26451.9940   
                 retiree                             22868.0050   
          M      business                            39192.0900   
                 civil servant                       31612.5500   
                 employee                            30974.4480   
                 retiree                             26554.6310   
<25       F      business                            23084.5230   
                 civil servant                       23611.6970   
                 employee                            19891.9970   
                 retiree                                    NaN   
          M      business                            27272.5915   
                 civil servant                       24617.5440   
                 employee                            27759.7065   
                 student                             15712.2600   
>66       F      business                            28124.2545   
                 civil servant                       57508.0320   
                 employee                            25137.4170   
                 retiree                             20897.4955   
          M      business                            40995.7380   
                 civil servant                       27687.3365   
                 employee                            27078.6405   
                 retiree                             21204.5365   

education                                     graduate degree  \
age_group gender income_type                                    
26-45     F      business                                 NaN   
                 civil servant                     17822.7570   
                 employee                                 NaN   
                 entrepreneur                             NaN   
                 paternity / maternity leave              NaN   
                 retiree                                  NaN   
                 unemployed                               NaN   
          M      business                                 NaN   
                 civil servant                            NaN   
                 employee                          25161.5835   
                 retiree                                  NaN   
                 unemployed                               NaN   
46-65     F      business                                 NaN   
                 civil servant                            NaN   
                 employee                                 NaN   
                 retiree                           40868.0310   
          M      business                                 NaN   
                 civil servant                            NaN   
                 employee                          42945.7940   
                 retiree                                  NaN   
<25       F      busine


All of these factors can influence and determine income. In order to accurately fill in missing values and perform a thorough analysis, I will consider all of them.

In [60]:
#  Write a function that we will use for filling in missing values
logs['total_income'] = logs['total_income'].fillna(logs.groupby(['age_group',
                                            'gender',
                                            'income_type',
                                            'education'])['total_income'].transform('median'))


In [61]:
# Check if it works
logs[logs.days_employed.isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,20604.1060,to have a wedding,46-65
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,28289.5110,education,26-45
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,18366.4590,building a real estate,46-65
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,20960.6690,second-hand car purchase,46-65
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,18366.4590,to have a wedding,46-65
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,24383.1285,transactions with commercial real estate,<25
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,22868.0050,purchase of the house for my family,46-65
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,35708.1140,transactions with commercial real estate,26-45
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,26451.9940,housing,46-65
83,0,NaN,52,secondary education,1,married,0,M,employee,0,24576.1275,housing,46-65


In [63]:
# Check if we got any errors
logs[logs.total_income.isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5880,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,46-65


In [64]:
logs[logs.income_type == 'entrepreneur']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5880,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,46-65
18450,0,520.848083,27,bachelor's degree,0,civil partnership,1,F,entrepreneur,0,79866.103,having a wedding,26-45


There is just two entrepreneurs in this dataset - not enough data for calculating accurate median income. For now I just fill missing income with median income relying on data that I have - age, education and gender. He is a 58-year-old man with a bachelor's degree.

In [65]:
logs_without_nan.pivot_table(index=['age_group', 'gender',], 
                             columns='education', 
                             values='total_income', 
                             aggfunc='median')

education         bachelor's degree  graduate degree  primary education  \
age_group gender                                                          
26-45     F              26244.5245       17822.7570         18384.1935   
          M              33919.5970       25161.5835         21925.2320   
46-65     F              27093.4390       40868.0310         17657.4995   
          M              32098.7200       42945.7940         18584.6300   
<25       F              21477.9765              NaN         17781.3660   
          M              27220.0380              NaN         27119.0240   
>66       F              21979.4380              NaN         13715.3340   
          M              24688.8660       15800.3990         21019.6355   

education         secondary education  some college  
age_group gender                                     
26-45     F                20607.6265    24715.7010  
          M                26240.0710    31599.9430  
46-65     F                20076.7770    25351.1100  
          M                24718.0370    30746.0770  
<25       F                17671.9595    19613.5050  
          M                23489.3950    25571.4005  
>66       F                16943.4920    28178.9170  
          M                18655.6060    19946.7950

In [66]:
# Replacing missing values if there are any errors
logs['total_income'] = logs['total_income'].fillna(32098)
logs[logs.income_type == 'entrepreneur']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5880,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,32098.000,buy residential real estate,46-65
18450,0,520.848083,27,bachelor's degree,0,civil partnership,1,F,entrepreneur,0,79866.103,having a wedding,26-45


In [67]:
# Checking the number of entries in the columns
logs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  int64  
 1   days_employed     19149 non-null  float64
 2   dob_years         21230 non-null  int64  
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  int64  
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  int64  
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  int64  
 10  total_income      21230 non-null  float64
 11  purpose           21230 non-null  object 
 12  age_group         21230 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Restoring values in `days_employed`


I think I would use the same factors as I did in the previous step: age, gender, income type and education.

In [68]:
# Distribution of `days_employed` medians based on your identified parameters
logs_without_nan.pivot_table(index=['age_group', 'gender', 'income_type'], 
                             columns='education', 
                             values='days_employed', 
                             aggfunc='median')


education                                     bachelor's degree  \
age_group gender income_type                                      
26-45     F      business                           1457.249197   
                 civil servant                      2383.288097   
                 employee                           1601.316234   
                 entrepreneur                        520.848083   
                 paternity / maternity leave                NaN   
                 retiree                          378335.092366   
                 unemployed                       395302.838654   
          M      business                           1172.242616   
                 civil servant                      2689.137274   
                 employee                           1329.454410   
                 retiree                          359159.819525   
                 unemployed                                 NaN   
46-65     F      business                           2100.158036   
                 civil servant                      4075.626656   
                 employee                           2377.947280   
                 retiree                          365826.463373   
          M      business                           1895.206419   
                 civil servant                      4141.630427   
                 employee                           1929.913775   
                 retiree                          364252.037932   
<25       F      business                            635.834283   
                 civil servant                      1179.796773   
                 employee                            712.646567   
                 retiree                                    NaN   
          M      business                            970.594155   
                 civil servant                      1206.015770   
                 employee                            628.251524   
                 student                             578.751554   
>66       F      business                           5268.462540   
                 civil servant                      6025.506521   
                 employee                           4448.911248   
                 retiree                          371697.508685   
          M      business                           2038.437948   
                 civil servant                      4487.652488   
                 employee                           4311.397186   
                 retiree                          366032.395290   

education                                     graduate degree  \
age_group gender income_type                                    
26-45     F      business                                 NaN   
                 civil servant                    5968.075884   
                 employee                                 NaN   
                 entrepreneur                             NaN   
                 paternity / maternity leave              NaN   
                 retiree                                  NaN   
                 unemployed                               NaN   
          M      business                                 NaN   
                 civil servant                            NaN   
                 employee                         1380.316041   
                 retiree                                  NaN   
                 unemployed                               NaN   
46-65     F      business                                 NaN   
                 civil servant                            NaN   
                 employee                                 NaN   
                 retiree                        376276.219531   
          M      business                                 NaN   
                 civil servant                            NaN   
                 employee                         5352.038180   
                 retiree                                  NaN   
<25       F      busine

As we remember the data on 'days_employed' for retirees is skewed. We need to redirect this issue to the data engeneers and ask them to rewiew the ways this data was acquierred. But for now the skewed data will not affect other values since I calculate median for every income type separetly.

In [69]:
# Distribution of `days_employed` means based on your identified parameters
logs_without_nan.pivot_table(index=['age_group', 'gender', 'income_type'], 
                             columns='education',  
                             values='days_employed', 
                             aggfunc='mean')


education                                     bachelor's degree  \
age_group gender income_type                                      
26-45     F      business                           1879.494887   
                 civil servant                      2765.453490   
                 employee                           2146.624018   
                 entrepreneur                        520.848083   
                 paternity / maternity leave                NaN   
                 retiree                          373771.949497   
                 unemployed                       395302.838654   
          M      business                           1650.750495   
                 civil servant                      2983.047724   
                 employee                           1900.601650   
                 retiree                          360320.508181   
                 unemployed                                 NaN   
46-65     F      business                           2978.821419   
                 civil servant                      4348.092139   
                 employee                           3539.384795   
                 retiree                          364927.794671   
          M      business                           2450.571345   
                 civil servant                      4536.331379   
                 employee                           2569.159386   
                 retiree                          365034.555409   
<25       F      business                            811.846775   
                 civil servant                      1293.917819   
                 employee                            855.000155   
                 retiree                                    NaN   
          M      business                           1138.938187   
                 civil servant                      1241.907765   
                 employee                            805.272449   
                 student                             578.751554   
>66       F      business                           5350.807156   
                 civil servant                      6025.506521   
                 employee                           5722.586865   
                 retiree                          368028.175796   
          M      business                           5695.053113   
                 civil servant                      4487.652488   
                 employee                           4311.397186   
                 retiree                          363044.994103   

education                                     graduate degree  \
age_group gender income_type                                    
26-45     F      business                                 NaN   
                 civil servant                    5968.075884   
                 employee                                 NaN   
                 entrepreneur                             NaN   
                 paternity / maternity leave              NaN   
                 retiree                                  NaN   
                 unemployed                               NaN   
          M      business                                 NaN   
                 civil servant                            NaN   
                 employee                         1380.316041   
                 retiree                                  NaN   
                 unemployed                               NaN   
46-65     F      business                                 NaN   
                 civil servant                            NaN   
                 employee                                 NaN   
                 retiree                        376276.219531   
          M      business                                 NaN   
                 civil servant                            NaN   
                 employee                         5352.038180   
                 retiree                                  NaN   
<25       F      busine


To avoid the influence of outliers, I will again use median values.

In [70]:
# Let's write a function that calculates means or medians (depending on your decision) 
#based on your identified parameter
logs['days_employed'] = logs['days_employed'].fillna(logs.groupby(['age_group',
                                            'gender',
                                            'income_type', 
                                            'education'])['days_employed'].transform('median'))

In [71]:
# Check that the function works
logs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  int64  
 1   days_employed     21229 non-null  float64
 2   dob_years         21230 non-null  int64  
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  int64  
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  int64  
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  int64  
 10  total_income      21230 non-null  float64
 11  purpose           21230 non-null  object 
 12  age_group         21230 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


In [72]:
logs[logs.days_employed.isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5880,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,32098.0,buy residential real estate,46-65


Same entrepreneur with not enough data to calculate reliable median for 'days_employed'. And again I just fill missing values with median relying on data that I have - age, education and gender.

In [73]:
logs_without_nan.pivot_table(index=['age_group', 'gender',], 
                             columns='education', 
                             values='days_employed', 
                             aggfunc='median')

education         bachelor's degree  graduate degree  primary education  \
age_group gender                                                          
26-45     F             1643.170124      5968.075884        1442.014009   
          M             1368.688217      1380.316041         971.256328   
46-65     F             4518.341388    376276.219531      348342.543651   
          M             2679.153060      5352.038180        4723.452006   
<25       F              756.310259              NaN         777.665808   
          M              754.060724              NaN         949.705772   
>66       F           363504.087447              NaN      360639.907161   
          M           351844.210409    337584.815560      347320.596996   

education         secondary education   some college  
age_group gender                                      
26-45     F               1707.080821    1296.932256  
          M               1500.308222    1309.112396  
46-65     F               9055.281279    3711.696343  
          M               2697.916973    2064.051724  
<25       F                862.528093     702.043712  
          M                775.899074     895.957814  
>66       F             361697.330514  375331.152368  
          M             362034.079738  352708.834270

In [74]:
logs['days_employed'] = logs['days_employed'].fillna(4518)

In [75]:
logs[logs.income_type == 'entrepreneur']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5880,0,4518.000000,58,bachelor's degree,0,married,0,M,entrepreneur,0,32098.000,buy residential real estate,46-65
18450,0,520.848083,27,bachelor's degree,0,civil partnership,1,F,entrepreneur,0,79866.103,having a wedding,26-45


In [79]:
# Check the entries in all columns - make sure we fixed all missing values
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  int64  
 1   days_employed     21230 non-null  float64
 2   dob_years         21230 non-null  int64  
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  int64  
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  int64  
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  int64  
 10  total_income      21230 non-null  float64
 11  purpose           21230 non-null  object 
 12  age_group         21230 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Yay! The dataset is fixed! Let's proseed!

## Categorization of data

As I noted before, the "purpose" column has some issues, with some values being written in different ways, such as "to have a wedding" and "having a wedding," or "buy real estate" and "purchase of the house."
I think this collumn need to be categorised.


In [80]:
# Print the values for your selected data for categorization
logs.purpose.value_counts()


wedding ceremony                            785
having a wedding                            759
to have a wedding                           755
real estate transactions                    669
buy commercial real estate                  655
buying property for renting out             647
transactions with commercial real estate    643
housing transactions                        641
purchase of the house for my family         636
housing                                     635
purchase of the house                       634
property                                    627
construction of own property                626
transactions with my real estate            623
building a property                         619
purchase of my own house                    618
building a real estate                      617
buy real estate                             612
housing renovation                          602
buy residential real estate                 599
buying my own car                       


Let's check unique values

In [81]:
# Check the unique values
logs.purpose.sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying a second-hand car',
       'buying my own car', 'buying property for renting out', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'housing transactions', 'profile education',
       'property', 'purchase of a car', 'purchase of my own house',
       'purchase of the house', 'purchase of the house for my family',
       'real estate transactions', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'transactions with commercial real estate',
       'transactions with my real estate', 'university education',
       'we

Main groups are: 
* real estate
* wedding
* education
* car

Based on these themes, we will categorize our data.


In [82]:
# Let's write a function to categorize the data based on common topics
categorization = {
    'building a property': 'real estate',
    'building a real estate': 'real estate',
    'buy commercial real estate': 'real estate',
    'buy real estate': 'real estate',
    'buy residential real estate': 'real estate',
    'buying a second-hand car': 'car',
    'buying my own car': 'car', 
    'buying property for renting out': 'real estate',
    'car': 'car',
    'car purchase': 'car', 
    'cars': 'car', 
    'construction of own property': 'real estate',
    'education': 'education', 
    'getting an education': 'education',  
    'getting higher education': 'education',
    'going to university': 'education', 
    'having a wedding': 'wedding',
    'housing': 'real estate',
    'housing renovation': 'real estate',
    'housing transactions': 'real estate',
    'profile education': 'education',
    'property': 'real estate',
    'purchase of a car': 'car', 
    'purchase of my own house': 'real estate',
    'purchase of the house': 'real estate',
    'purchase of the house for my family': 'real estate',
    'real estate transactions': 'real estate',
    'second-hand car purchase': 'car',
    'supplementary education': 'education', 
    'to become educated': 'education', 
    'to buy a car': 'car',
    'to get a supplementary education': 'education', 
    'to have a wedding': 'wedding',
    'to own a car': 'car', 
    'transactions with commercial real estate': 'real estate',
    'transactions with my real estate': 'real estate',
    'university education': 'education',
    'wedding ceremony': 'wedding'
}

logs['purpose_category'] = logs['purpose'].replace(categorization)

In [83]:
# Create a column with the categories and count the values for them
logs.purpose_category.value_counts()

real estate    10703
car             4258
education       3970
wedding         2299
Name: purpose_category, dtype: int64

In [84]:
logs.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,26-45,real estate
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,26-45,car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,26-45,real estate
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,26-45,education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,46-65,wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,26-45,real estate
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,26-45,real estate
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,46-65,education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,26-45,wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,26-45,real estate


Let's also categorize total income

In [85]:
# Looking through all the numerical data in your selected column for categorization
logs.total_income.value_counts()

19898.7265    284
18366.4590    221
25321.8700    186
20161.9755    163
23051.3540    104
             ... 
30698.4310      1
24143.6710      1
23879.6790      1
46127.6210      1
41428.9160      1
Name: total_income, Length: 19188, dtype: int64

In [86]:
# Getting summary statistics for the column
logs.total_income.describe().astype(int)


count     21230
mean      26489
std       15788
min        3306
25%       17096
50%       23143
75%       31570
max      362496
Name: total_income, dtype: int64

In [87]:
logs.total_income.median()

23143.0115

As there wasn't provided sensitive information about context of the data I cannot come up with meaningful categorisation for income. 
I have nothing to reference while creating the categories except for summary statistics.
 
I will use following categories:

* low income, less than 20k

* mid-low income, 20-50k

* mid-high income, 50-100k

* high income, 100k+

In [88]:
# Creating function for categorizing into different numerical groups based on ranges
def total_income_level(total_income):
    if (total_income > 0) and (total_income <= 20000):
        return 'low'
    if (total_income > 20000) and (total_income <= 50000):
        return 'mid-low'
    if (total_income > 50000) and (total_income <= 100000):
        return 'mid-high'
    if (total_income > 100000):
        return 'high'
    else:
        return 'error'


In [89]:
total_income_level(55000)

'mid-high'

In [90]:
# Creating column with categories
logs['total_income_level'] = logs['total_income'].apply(total_income_level)

In [91]:
# Count each categories values to see the distribution
logs['total_income_level'].value_counts()

mid-low     11971
low          7945
mid-high     1215
high           99
Name: total_income_level, dtype: int64

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [93]:
# Check the children data and paying back on time
pivot_table_children_default = logs.pivot_table(index='children', 
                                               columns='debt', 
                                               values='days_employed', 
                                               aggfunc='count')


# Calculating default-rate based on the number of children

pivot_table_children_default['percentage'] = pivot_table_children_default[1] / (pivot_table_children_default[1]+pivot_table_children_default[0])*100
pivot_table_children_default.sort_values(by='percentage')

debt,0,1,percentage
children,,,
0,12963.0,1058.0,7.545824
3,301.0,27.0,8.231707
1,4351.0,441.0,9.202838
2,1845.0,194.0,9.514468
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


**Conclusion**

The default rate is not significantly affected by the number of children a person has, but rather by the fact of having children. In fact, the lowest default rate is seen among those without children.

**Is there a correlation between family status and paying back on time?**

In [94]:
# Check the family status data and paying back on time
pivot_table_family_default = logs.pivot_table(index='family_status', 
                                               columns='debt', 
                                               values='days_employed', 
                                               aggfunc='count')



# Calculating default-rate based on family status
pivot_table_family_default['percentage'] = pivot_table_family_default[1] / (pivot_table_family_default[1]+pivot_table_family_default[0])*100
pivot_table_family_default.sort_values(by='percentage')


debt,0,1,percentage
family_status,,,
widow / widower,884,62,6.553911
divorced,1095,84,7.124682
married,11290,923,7.557521
civil partnership,3729,383,9.314202
unmarried,2508,272,9.784173


**Conclusion**

The default rate is lower among people who have been or are currently in a legally recognized marriage compared to those who have never been in one.

**Is there a correlation between income level and paying back on time?**

In [95]:
# Check the income level data and paying back on time
pivot_table_income_default = logs.pivot_table(index='total_income_level', 
                                               columns='debt', 
                                               values='days_employed', 
                                               aggfunc='count')


# Calculating default-rate based on income level
pivot_table_income_default['percentage'] = pivot_table_income_default[1] / (pivot_table_income_default[1]+pivot_table_income_default[0])*100
pivot_table_income_default.sort_values(by='percentage')


debt,0,1,percentage
total_income_level,,,
high,93,6,6.060606
mid-high,1129,86,7.078189
mid-low,11004,967,8.077855
low,7280,665,8.370044


**Conclusion**

People with low income level is more likely to default. And people with high income level is less likely to default. 


But it should be noted that no context was provided with the dataset, so I have nothing to reference while creating the categories except for summary statistics.


**How does credit purpose affect the default rate?**

In [96]:
# Check the percentages for default rate for each credit purpose and analyze them
pivot_table_purpose_default = logs.pivot_table(index='purpose_category', 
                                               columns='debt', 
                                               values='days_employed', 
                                               aggfunc='count')

pivot_table_purpose_default['percentage'] = pivot_table_purpose_default[1] / (pivot_table_purpose_default[1]+pivot_table_purpose_default[0])*100
pivot_table_purpose_default.sort_values(by='percentage')


debt,0,1,percentage
purpose_category,,,
real estate,9926,777,7.259647
wedding,2118,181,7.872988
education,3601,369,9.294710
car,3861,397,9.323626


**Conclusion**

People who take out loans for education or car-related expenses are more likely to default on their debt.

# General Conclusion 

Here are my findings:

1. The impact of the number of children on the default rate is not significant, but having children does increase the default rate. The lowest default rate is seen among those without children.
2. The default rate is lower among people who have been or are currently in a legally recognized marriage compared to those who have never been in one.
3. People with low income levels are more likely to default on their loans, while those with high income levels are less likely to default.
4. People who take out loans for education or car-related expenses are more likely to default on their debt.


During the data analysis process, I encountered several issues that should be discussed with data engineers:

* A large number of missing values may have occurred during the data acquisition process (in columns describing income and work time).
* Illogical values may have been introduced during the data entry process.
* As a result of the previous two issues, it was not possible to analyze data on one category of income type - retirees.
